In [1]:
!nodetool status

Datacenter: datacenter1
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address     Load        Tokens  Owns (effective)  Host ID                               Rack 
UN  172.30.0.4  180.11 KiB  16      100.0%            42284706-aa54-4536-b6bd-0379cd791325  rack1
UN  172.30.0.2  179.9 KiB   16      100.0%            4a3d5237-4ce4-4582-ad8c-09da30b557cb  rack1
UN  172.30.0.3  160.24 KiB  16      100.0%            39624122-273e-47cd-8ae9-051ce76bb9cb  rack1



In [2]:
!nodetool ring


Datacenter: datacenter1
Address          Rack        Status State   Load            Owns                Token                                       
                                                                                9136487762688204160                         
172.30.0.3       rack1       Up     Normal  160.24 KiB      100.00%             -8880207632489486139                        
172.30.0.4       rack1       Up     Normal  180.11 KiB      100.00%             -8842701675031596006                        
172.30.0.2       rack1       Up     Normal  179.9 KiB       100.00%             -8230883503573745797                        
172.30.0.3       rack1       Up     Normal  160.24 KiB      100.00%             -7808707016149453326                        
172.30.0.4       rack1       Up     Normal  180.11 KiB      100.00%             -7599181669351004032                        
172.30.0.2       rack1       Up     Normal  179.9 KiB       100.00%             -6987363497893153823

In [3]:
!cqlsh cassandra-db-1 -e help


Documented shell commands:
CAPTURE  CLS          COPY  DESCRIBE  EXPAND  LOGIN   SERIAL  SOURCE   UNICODE
CLEAR    CONSISTENCY  DESC  EXIT      HELP    PAGING  SHOW    TRACING

CQL help topics:
AGGREGATES               CREATE_KEYSPACE           DROP_TRIGGER      TEXT     
ALTER_KEYSPACE           CREATE_MATERIALIZED_VIEW  DROP_TYPE         TIME     
ALTER_MATERIALIZED_VIEW  CREATE_ROLE               DROP_USER         TIMESTAMP
ALTER_TABLE              CREATE_TABLE              FUNCTIONS         TRUNCATE 
ALTER_TYPE               CREATE_TRIGGER            GRANT             TYPES    
ALTER_USER               CREATE_TYPE               INSERT            UPDATE   
APPLY                    CREATE_USER               INSERT_JSON       USE      
ASCII                    DATE                      INT               UUID     
BATCH                    DELETE                    JSON            
BEGIN                    DROP_AGGREGATE            KEYWORDS        
BLOB                     DROP_COLUMNF

In [4]:
!cqlsh cassandra-db-1 -e "describe keyspaces"


banking  system_auth         system_schema  system_views         
system   system_distributed  system_traces  system_virtual_schema



In [5]:
# !cqlsh cassandra-db-1 -e "use banking"

In [5]:
!cqlsh cassandra-db-1 -e "describe tables"


Keyspace banking
----------------
loans

Keyspace system
---------------
available_ranges     paxos_repair_history  sstable_activity       
available_ranges_v2  peer_events           sstable_activity_v2    
batches              peer_events_v2        table_estimates        
built_views          peers                 top_partitions         
compaction_history   peers_v2              transferred_ranges     
"IndexInfo"          prepared_statements   transferred_ranges_v2  
local                repairs               view_builds_in_progress
paxos                size_estimates      

Keyspace system_auth
--------------------
network_permissions             role_members      roles
resource_role_permissons_index  role_permissions

Keyspace system_distributed
---------------------------
parent_repair_history  partition_denylist  repair_history  view_build_status

Keyspace system_schema
----------------------
aggregates  dropped_columns  indexes    tables    types
columns     functions        k

In [6]:
from cassandra.cluster import Cluster
cluster = Cluster(["cassandra-db-1", "cassandra-db-2", "cassandra-db-3"])
cass = cluster.connect()

In [7]:
# cass.execute("create keyspace banking with replication={'class': 'SimpleStrategy', 'replication_factor': 2};")

In [8]:
cass.execute("use banking")

In [9]:
cass.execute("drop table if exists loans")

In [10]:
cass.execute("""
CREATE TABLE loans (
    bank_id INT,
    bank_name TEXT STATIC,
    loan_id UUID,
    amount INT,
    state TEXT,
    PRIMARY KEY ((bank_id), amount, loan_id)
) WITH CLUSTERING ORDER BY (amount DESC, loan_id ASC)
""")

In [11]:
print(cass.execute("describe table loans").one().create_statement)

CREATE TABLE banking.loans (
    bank_id int,
    amount int,
    loan_id uuid,
    bank_name text static,
    state text,
    PRIMARY KEY (bank_id, amount, loan_id)
) WITH CLUSTERING ORDER BY (amount DESC, loan_id ASC)
    AND additional_write_policy = '99p'
    AND bloom_filter_fp_chance = 0.01
    AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}
    AND cdc = false
    AND comment = ''
    AND compaction = {'class': 'org.apache.cassandra.db.compaction.SizeTieredCompactionStrategy', 'max_threshold': '32', 'min_threshold': '4'}
    AND compression = {'chunk_length_in_kb': '16', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}
    AND memtable = 'default'
    AND crc_check_chance = 1.0
    AND default_time_to_live = 0
    AND extensions = {}
    AND gc_grace_seconds = 864000
    AND max_index_interval = 2048
    AND memtable_flush_period_in_ms = 0
    AND min_index_interval = 128
    AND read_repair = 'BLOCKING'
    AND speculative_retry = '99p';


In [12]:
# INSERT is really UPSERT (meaning update OR insert)
cass.execute("""
INSERT INTO loans (bank_id, bank_name)
VALUES (544, 'test2')
""")

In [13]:
import pandas as pd
pd.DataFrame(cass.execute("select * from loans"))

,bank_id,amount,loan_id,bank_name,state
0,544,None,None,test2,None


In [14]:
cass.execute("""
INSERT INTO loans (bank_id, bank_name, loan_id)
VALUES (544, 'test2', UUID())
""")

InvalidRequest: Error from server: code=2200 [Invalid query] message="Some clustering keys are missing: amount"

In [15]:
cass.execute("""
INSERT INTO loans (bank_id, amount, loan_id)
VALUES (544, 300, UUID())
""")

In [16]:
pd.DataFrame(cass.execute("select * from loans"))

,bank_id,amount,loan_id,bank_name,state
0,544,300,108214fc-ad20-420e-8430-6a7ed5fa4d69,test2,None


In [17]:
# UUID() and NOW() both return a UUID, 
# but NOW() does better at being unique
# NOW () looks at MAC addr, timestamp, seq num

In [18]:
cass.execute("""
INSERT INTO loans (bank_id, bank_name, amount, loan_id, state)
VALUES (544, 'mybank', 400, NOW(), 'wi')
""")

In [19]:
pd.DataFrame(cass.execute("select * from loans"))

,bank_id,amount,loan_id,bank_name,state
0,544,400,80f3b9d0-b15f-11ee-bb06-5384c159d34b,mybank,wi
1,544,300,108214fc-ad20-420e-8430-6a7ed5fa4d69,mybank,None


In [20]:
cass.execute("""
INSERT INTO loans (bank_id, bank_name, amount, loan_id, state)
VALUES (999, 'uwcu', 500, NOW(), 'il')
""")

In [21]:
pd.DataFrame(cass.execute("select * from loans"))

,bank_id,amount,loan_id,bank_name,state
0,544,400,80f3b9d0-b15f-11ee-bb06-5384c159d34b,mybank,wi
1,544,300,108214fc-ad20-420e-8430-6a7ed5fa4d69,mybank,None
2,999,500,82d54b10-b15f-11ee-b3cd-bb68e247982e,uwcu,il


In [22]:
cass.execute("""
create type FullName (
    first TEXT,
    last TEXT
)
""")

InvalidRequest: Error from server: code=2200 [Invalid query] message="A user type with name 'fullname' already exists"

In [23]:
# fast, because we have sparse table
cass.execute("""
alter table loans add (username FullName)
""")

In [24]:
cass.execute("""
INSERT INTO loans (bank_id, bank_name, amount, loan_id, username)
VALUES (999, 'uwcu', 500, NOW(), {first: 'Tyler', last: 'Caraza-Harter'})
""")

In [25]:
cass.execute("""
INSERT INTO loans (bank_id, bank_name, amount, loan_id, username)
VALUES (999, 'uwcu', 500, NOW(), {first: 'onlyfirst'})
""")

In [26]:
pd.DataFrame(cass.execute("select username, username.first, username.last from loans"))

,username,username_first,username_last
0,None,None,None
1,None,None,None
2,None,None,None
3,"(Tyler, Caraza-Harter)",Tyler,Caraza-Harter
4,"(onlyfirst, None)",onlyfirst,None


# Prepared Statements

In [27]:
insert_uwcu = cass.prepare("""
INSERT INTO loans (bank_id, bank_name, amount, loan_id, username)
VALUES (999, 'uwcu', ?, NOW(), {first: ?, last: ?})
""")

In [28]:
# insert_uwcu.VARIOUS_CONFIG

In [29]:
cass.execute(insert_uwcu, (301, "TestFirst", "TestLast"))

In [31]:
pd.DataFrame(cass.execute("select * from loans"))

,bank_id,amount,loan_id,bank_name,state,username
0,544,400,80f3b9d0-b15f-11ee-bb06-5384c159d34b,mybank,wi,None
1,544,300,108214fc-ad20-420e-8430-6a7ed5fa4d69,mybank,None,None
2,999,500,82d54b10-b15f-11ee-b3cd-bb68e247982e,uwcu,il,None
3,999,500,85dc97a0-b15f-11ee-bb06-5384c159d34b,uwcu,None,"(Tyler, Caraza-Harter)"
4,999,500,85e7bb30-b15f-11ee-bb36-1badc6008a84,uwcu,None,"(onlyfirst, None)"
5,999,301,886872a0-b15f-11ee-bb36-1badc6008a84,uwcu,None,"(TestFirst, TestLast)"


In [32]:
pd.DataFrame(cass.execute("""
SELECT bank_id, bank_name, AVG(amount) 
FROM loans 
GROUP BY bank_id
"""))

,bank_id,bank_name,system_avg_amount
0,544,mybank,350
1,999,uwcu,450


In [33]:
# can only group by partition (Cassandra limitation)
# pd.DataFrame(cass.execute("""
# SELECT state, AVG(amount) 
# FROM loans 
# GROUP BY state
# """))

In [34]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder
         .appName("cs544")
         .config('spark.jars.packages', 'com.datastax.spark:spark-cassandra-connector_2.12:3.4.0')
         .config("spark.sql.extensions", "com.datastax.spark.connector.CassandraSparkExtensions")
         .getOrCreate())

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-3137f25e-3dda-42de-b1f7-7c844df19c46;1.0
	confs: [default]
	found com.datastax.spark#spark-cassandra-connector_2.12;3.4.0 in central
	found com.datastax.spark#spark-cassandra-connector-driver_2.12;3.4.0 in central
	found com.datastax.oss#java-driver-core-shaded;4.13.0 in central
	found com.datastax.oss#native-protocol;1.5.0 in central
	found com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 in central
	found com.typesafe#config;1.4.1 in central
	found org.slf4j#slf4j-api;1.7.26 in central
	found io.dropwizard.metrics#metrics-core;4.1.18 in central
	found org.hdrhistogram#HdrHistogram;2.1.12 in central
	found org.reactivestreams#reactive-streams;1.0.3 in central
	found com.github.stephenc.jcip#jcip-annotations;1.0-1 in central
	found com.gith

In [35]:
# catalog name `mycat`, i can change that
spark.conf.set("spark.sql.catalog.mycat", 
               "com.datastax.spark.connector.datasource.CassandraCatalog")
spark.conf.set("spark.sql.catalog.mycat.spark.cassandra.connection.host", 
               "cassandra-db-1,cassandra-db-2,cassandra-db-3")

In [36]:
spark.sql("""
SELECT *
FROM mycat.banking.loans
""")

DataFrame[bank_id: int, amount: int, loan_id: string, state: string, username: struct<first:string,last:string>, bank_name: string]

In [37]:
spark.sql("""
SELECT *
FROM mycat.banking.loans
""").toPandas()

,bank_id,amount,loan_id,state,username,bank_name
0,544,400,80f3b9d0-b15f-11ee-bb06-5384c159d34b,wi,None,mybank
1,544,300,108214fc-ad20-420e-8430-6a7ed5fa4d69,None,None,mybank
2,999,500,82d54b10-b15f-11ee-b3cd-bb68e247982e,il,None,uwcu
3,999,500,85dc97a0-b15f-11ee-bb06-5384c159d34b,None,"(Tyler, Caraza-Harter)",uwcu
4,999,500,85e7bb30-b15f-11ee-bb36-1badc6008a84,None,"(onlyfirst, None)",uwcu
5,999,301,886872a0-b15f-11ee-bb36-1badc6008a84,None,"(TestFirst, TestLast)",uwcu


In [38]:
spark.sql("""
select state, AVG(amount) 
from mycat.banking.loans
GROUP BY state
""").toPandas()

,state,avg(amount)
0,il,500.00
1,None,400.25
2,wi,400.00


In [39]:
# dump OLTP to parquet in HDFS, or, Hive, or wherever, and 
# use for OLAP
# spark.sql("""
# SELECT *
# FROM mycat.banking.loans
# """).write...

In [40]:
# Spark is great from ETL pipelines

cassandra goal is highly available,
    boss nodes are single point of failure
    
whereas HBase are BigTable are highly reliable but not available

Partitioning 

Elasticity

Consistent Hashing

merkle tree

# Hash Partitioning Example: Not Elastic

In [45]:
import pandas as pd
import string

In [46]:
string.ascii_uppercase

'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [47]:
df = pd.DataFrame({"letter": list(string.ascii_uppercase)})
df.head()

,letter
0,A
1,B
2,C
3,D
4,E


In [48]:
df["partition-before"] = df["letter"].apply(lambda letter: hash(letter) % 4)
df.head()

,letter,partition-before
0,A,2
1,B,3
2,C,2
3,D,1
4,E,0


In [49]:
df["partition-after"] = df["letter"].apply(lambda letter: hash(letter) % 5)
df.head()

,letter,partition-before,partition-after
0,A,2,0
1,B,3,1
2,C,2,2
3,D,1,2
4,E,0,3


In [51]:
# only 15.3% stayed with the same partition number
(df["partition-before"] == df["partition-after"]).mean()

0.15384615384615385

# https://mediaspace.wisc.edu/media/Tyler%20Caraza-Harter-Noland%20132-11_08_23-13%3A18%3A45/1_0785i0x0

In [42]:
!nodetool ring


Datacenter: datacenter1
Address          Rack        Status State   Load            Owns                Token                                       
                                                                                9136487762688204160                         
172.30.0.3       rack1       Up     Normal  274 KiB         100.00%             -8880207632489486139                        
172.30.0.4       rack1       Up     Normal  260.09 KiB      100.00%             -8842701675031596006                        
172.30.0.2       rack1       Up     Normal  259.35 KiB      100.00%             -8230883503573745797                        
172.30.0.3       rack1       Up     Normal  274 KiB         100.00%             -7808707016149453326                        
172.30.0.4       rack1       Up     Normal  260.09 KiB      100.00%             -7599181669351004032                        
172.30.0.2       rack1       Up     Normal  259.35 KiB      100.00%             -6987363497893153823

In [43]:
!nodetool ring | wc

     58     420    6433


In [46]:
!cqlsh cassandra-db-1 -e "use banking;select * from loans;"


 bank_id | amount | loan_id                              | bank_name | state | username
---------+--------+--------------------------------------+-----------+-------+-----------------------------------------
     544 |    400 | 80f3b9d0-b15f-11ee-bb06-5384c159d34b |    mybank |    wi |                                    null
     544 |    300 | 108214fc-ad20-420e-8430-6a7ed5fa4d69 |    mybank |  null |                                    null
     999 |    500 | 82d54b10-b15f-11ee-b3cd-bb68e247982e |      uwcu |    il |                                    null
     999 |    500 | 85dc97a0-b15f-11ee-bb06-5384c159d34b |      uwcu |  null | {first: 'Tyler', last: 'Caraza-Harter'}
     999 |    500 | 85e7bb30-b15f-11ee-bb36-1badc6008a84 |      uwcu |  null |        {first: 'onlyfirst', last: null}
     999 |    301 | 886872a0-b15f-11ee-bb36-1badc6008a84 |      uwcu |  null |  {first: 'TestFirst', last: 'TestLast'}

(6 rows)


In [47]:
!cqlsh cassandra-db-1 -e "select * from banking.loans;"


 bank_id | amount | loan_id                              | bank_name | state | username
---------+--------+--------------------------------------+-----------+-------+-----------------------------------------
     544 |    400 | 80f3b9d0-b15f-11ee-bb06-5384c159d34b |    mybank |    wi |                                    null
     544 |    300 | 108214fc-ad20-420e-8430-6a7ed5fa4d69 |    mybank |  null |                                    null
     999 |    500 | 82d54b10-b15f-11ee-b3cd-bb68e247982e |      uwcu |    il |                                    null
     999 |    500 | 85dc97a0-b15f-11ee-bb06-5384c159d34b |      uwcu |  null | {first: 'Tyler', last: 'Caraza-Harter'}
     999 |    500 | 85e7bb30-b15f-11ee-bb36-1badc6008a84 |      uwcu |  null |        {first: 'onlyfirst', last: null}
     999 |    301 | 886872a0-b15f-11ee-bb36-1badc6008a84 |      uwcu |  null |  {first: 'TestFirst', last: 'TestLast'}

(6 rows)


In [48]:
!cqlsh cassandra-db-1 -e "select bank_id, token(bank_id) from banking.loans;"


 bank_id | system.token(bank_id)
---------+-----------------------
     544 |  -2546916261887148350
     544 |  -2546916261887148350
     999 |   7071430368280192841
     999 |   7071430368280192841
     999 |   7071430368280192841
     999 |   7071430368280192841

(6 rows)


In [50]:
!cqlsh cassandra-db-1 -e "select bank_id, writetime(bank_id) from banking.loans;"

<stdin>:1:InvalidRequest: Error from server: code=2200 [Invalid query] message="Cannot use selection function writeTime on PRIMARY KEY part bank_id"


In [52]:
!cqlsh cassandra-db-1 -e "select state, writetime(state) from banking.loans;"


 state | writetime(state)
-------+------------------
    wi | 1705073423846821
  null |             null
    il | 1705073427006343
  null |             null
  null |             null
  null |             null

(6 rows)


In [53]:
!cqlsh cassandra-db-1 -e "update banking.loans set state='wi' WHERE bank_id=544 AND amount=400 AND loan_id=80f3b9d0-b15f-11ee-bb06-5384c159d34b;"

In [54]:
!cqlsh cassandra-db-1 -e "select * from banking.loans;"


 bank_id | amount | loan_id                              | bank_name | state | username
---------+--------+--------------------------------------+-----------+-------+-----------------------------------------
     544 |    400 | 80f3b9d0-b15f-11ee-bb06-5384c159d34b |    mybank |    wi |                                    null
     544 |    300 | 108214fc-ad20-420e-8430-6a7ed5fa4d69 |    mybank |  null |                                    null
     999 |    500 | 82d54b10-b15f-11ee-b3cd-bb68e247982e |      uwcu |    il |                                    null
     999 |    500 | 85dc97a0-b15f-11ee-bb06-5384c159d34b |      uwcu |  null | {first: 'Tyler', last: 'Caraza-Harter'}
     999 |    500 | 85e7bb30-b15f-11ee-bb36-1badc6008a84 |      uwcu |  null |        {first: 'onlyfirst', last: null}
     999 |    301 | 886872a0-b15f-11ee-bb36-1badc6008a84 |      uwcu |  null |  {first: 'TestFirst', last: 'TestLast'}

(6 rows)
